# Problem setting

In this tutorial, we demonstrate how graph neural networks can be used for recommendation. Here we focus on item-based recommendation model. This method in this tutorial recommends items that are similar to the ones purchased by the user. We demonstrate the recommendation model on the MovieLens dataset.

# Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works with Pytorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL, we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [ ]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Load the rest of necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse as spsp

## Load MovieLens from pickle

In [ ]:
import pickle
user_movie_spm = pickle.load(open('movielens/movielens_orig_train.pkl', 'rb'))
features = pickle.load(open('movielens/movielens_features.pkl', 'rb'))
valid_set, test_set = pickle.load(open('movielens/movielens_eval.pkl', 'rb'))
neg_valid, neg_test = pickle.load(open('movielens/movielens_neg.pkl', 'rb'))

num_users = user_movie_spm.shape[0]
num_movies = user_movie_spm.shape[1]

users_valid = np.arange(num_users)
movies_valid = valid_set
users_test = np.arange(num_users)
movies_test = test_set

movie_popularity = user_movie_spm.transpose().dot(np.ones(shape=(num_users)))
# We need to rescale the values
movie_popularity = torch.tensor(movie_popularity / np.max(movie_popularity), dtype=torch.float32).unsqueeze(1)

u, s, vt = spsp.linalg.svds(user_movie_spm)
v = torch.tensor(vt.transpose(), dtype=torch.float32)
v = v * torch.tensor(np.sqrt(s).transpose(), dtype=torch.float32)

#features = torch.cat([features, movie_popularity, v], 1)
one_hot = torch.tensor(np.diag(np.ones(shape=(num_movies))), dtype=torch.float32)
features = torch.cat([features, one_hot], 1)
in_feats = features.shape[1]
print('#feats:', in_feats)

## Load BookCrossing from pickle

In [ ]:
import pickle
user_movie_spm = pickle.load(open('bx/bx_train.pkl', 'rb'))
abstracts = pickle.load(open('bx/bx_book_abstract.pkl', 'rb'))
titles = pickle.load(open('bx/bx_book_title.pkl', 'rb'))
features = torch.tensor(np.concatenate((titles, abstracts), 1), dtype=torch.float32)
valid_set, test_set = pickle.load(open('bx/bx_eval.pkl', 'rb'))
neg_valid, neg_test = pickle.load(open('bx/bx_neg.pkl', 'rb'))

num_users = user_movie_spm.shape[0]
num_movies = user_movie_spm.shape[1]

users_valid = np.arange(num_users, dtype=np.int64)
movies_valid = valid_set
users_test = np.arange(num_users, dtype=np.int64)
movies_test = test_set

in_feats = features.shape[1]
print('#feats:', in_feats)

Find the items watched/read/used by users in the testing set and their popularity.

In [ ]:
movie_deg = user_movie_spm.transpose().dot(np.ones((num_users)))
test_deg = np.zeros((num_users))
for i in range(num_users):
    movie = int(movies_test[i])
    test_deg[i] = movie_deg[movie]
test_deg_dict = {}
for i in range(1, 10):
    test_deg_dict[i] = np.nonzero(test_deg == i)[0]
for i in range(1, 10):
    test_deg_dict[i*10] = np.nonzero(np.logical_and(i*10 <= test_deg, test_deg < (i+1)*10))[0]
test_deg_dict[100] = np.nonzero(test_deg >= 100)[0]
tmp = 0
for key, deg in test_deg_dict.items():
    print(key, len(deg))
    tmp += len(deg)
print(num_users, tmp)

## Load Yelp2018 from pickle

In [ ]:
import pickle
user_movie_spm = pickle.load(open('yelp/yelp2018_orig_train.pkl', 'rb'))
features = pickle.load(open('yelp/yelp2018_entity_embed_features.pkl', 'rb'))
valid_set, test_set = pickle.load(open('yelp/yelp2018_eval.pkl', 'rb'))
neg_valid, neg_test = pickle.load(open('yelp/yelp2018_neg.pkl', 'rb'))

num_users = user_movie_spm.shape[0]
num_movies = user_movie_spm.shape[1]

users_valid = np.arange(num_users, dtype=np.int64)
movies_valid = valid_set
users_test = np.arange(num_users, dtype=np.int64)
movies_test = test_set

in_feats = features.shape[1]
print('#feats:', in_feats)

In [ ]:
print(user_movie_spm.shape)
print(user_movie_spm.nnz)

# The recommendation model

At large, the model first learns item embeddings from the user-item interaction dataset and use the item embeddings to recommend users similar items they have purchased. To learn item embeddings, we first need to construct an item similarity graph and train GNN on the item graph.

There are many ways of constructing the item similarity graph. Here we use the [SLIM model](https://dl.acm.org/citation.cfm?id=2118303) to learn item similarity and use the learned result to construct the item graph. The resulting graph will have an edge between two items if they are similar and the edge has a weight that represents the similarity score.

After the item similarity graph is constructed, we run a GNN model on it and use the vertex connectivity as the training signal to train the GNN model. The GNN training procedure is very similar to the link prediction task in [the previous section](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

## Construct the movie graph with SLIM
SLIM is an item-based recommendation model. When training SLIM on a user-item dataset, it learns an item similarity graph. This similarity graph is the item graph we construct for the GNN model.

Please follow the instruction on the [SLIM github repo](https://github.com/KarypisLab/SLIM) to install SLIM.

The SLIM only needs to run once and can be saved on the disk for future experiments.

In [ ]:
from SLIM import SLIM, SLIMatrix
model = SLIM()
params = {'algo': 'cd', 'nthreads': 32, 'l1r': 0.1, 'l2r': 2000}
trainmat = SLIMatrix(user_movie_spm.tocsr())
model.train(params, trainmat)
#model.save_model(modelfname='slim_model.csr', mapfname='slim_map.csr')

Evaluate the performance of the SLIM model.

In [ ]:
candidates = {}
neg_sample_size = neg_test.shape[1]
for i, neg_items in enumerate(neg_test):
    candidates[i] = np.zeros(shape=(neg_items.shape[0] + 1,), dtype=np.int64)
    candidates[i][:neg_sample_size] = neg_items
    candidates[i][-1] = int(movies_test[i])
rcmd_list = model.predict(trainmat, nrcmds=10, negitems=candidates, nnegs=100)
rcmd_mat = np.ones(shape=(num_users, len(rcmd_list[0])), dtype=np.int64)
for i, rcmd in rcmd_list.items():
    rcmd_mat[i] = rcmd

def test(rcmd_mat, true_list):
    hits = 0
    true_list = np.expand_dims(true_list, 1)
    hits = np.sum(rcmd_mat == true_list)
    #for u, rlist in rcmd_list.items():
    #    hits += true_list[u] in rlist
    print("Number of test(valid) users: %d(%d), hits@%d: %.4f" % 
          (len(true_list), len(rcmd_list), len(rcmd_list[0]), hits / len(rcmd_list)))
    return hits / len(rcmd_list)
test(rcmd_mat, movies_test)

Load the SLIM similarity matrix into DGL. We store the vertex similarity as edge data on DGL.

In [ ]:
from slim_load import read_csr

movie_spm = read_csr('slim_model.csr')
print('#edges:', movie_spm.nnz)
print('most similar:', np.max(movie_spm.data))
print('most unsimilar:', np.min(movie_spm.data))

In [ ]:
deg = movie_spm.dot(np.ones((num_movies)))
print(np.sum(deg == 0))
print(len(deg))
print(movie_spm.sum(0))

In [ ]:
g = dgl.DGLGraph(movie_spm, readonly=True)
g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
g.ndata['feats'] = features

## Construct the co-watch graph

In [ ]:
user_movie_spm = user_movie_spm.tocoo()
user_id = user_movie_spm.row
movie_id = user_movie_spm.col
movie_deg = user_movie_spm.transpose().dot(np.ones((num_users,)))
movie_ratio = movie_deg / np.sum(movie_deg)
# 1e-6 is a hyperparameter for this dataset.
movie_sample_prob = 1 - np.maximum(1 - np.sqrt(1e-5 / movie_ratio), 0)
sample_prob = movie_sample_prob[movie_id]
sample = np.random.uniform(size=(len(movie_id),))
user_id = user_id[sample_prob > sample]
movie_id = movie_id[sample_prob > sample]
print('#samples:', len(user_id))
spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
print(spm.shape)
movie_deg = spm.transpose().dot(np.ones((num_users,)))
print(np.sum(movie_deg == 0))

Find the top co-watched movie pairs.

In [ ]:
movie_spm = np.dot(spm.transpose(), spm)
print(movie_spm.nnz)
dense_movie = np.sort(movie_spm.todense())
topk_movie = dense_movie[:,-50]
topk_movie_spm = movie_spm > topk_movie
topk_movie_spm = spsp.csr_matrix(topk_movie_spm)

Find the movie pairs with top K similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
movie_spm = cosine_similarity(user_movie_spm.transpose(),dense_output=False)

dense_movie = np.sort(movie_spm.todense())
topk_movie = dense_movie[:,-30]
topk_movie_spm = movie_spm > topk_movie
topk_movie_spm = spsp.csr_matrix(topk_movie_spm)
topk_movie_spm = movie_spm.multiply(topk_movie_spm)

Construct the DGL graph with movie similarity.

In [ ]:
print(topk_movie_spm.nnz)
g = dgl.DGLGraph(topk_movie_spm, readonly=True)
g.edata['similarity'] = torch.tensor(topk_movie_spm.data)
g.ndata['feats'] = features
#g.ndata['feats'] = torch.arange(num_movies, dtype=torch.int64)

In [ ]:
g = dgl.DGLGraph(movie_spm, readonly=True)
g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
g.ndata['feats'] = features

## GNN models

We run GNN on the item graph to compute item embeddings. In this tutorial, we use a customized [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) model to compute node embeddings. The original GraphSage performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.

The original GraphSage model treats each neighbor equally. However, the SLIM model learns the item similarity based on the user-item iteration. The GNN model should take the similarity into account. Thus, we customize the GraphSage model in the following fashion. Instead of aggregating all neighbors equally, we aggregate neighbors embeddings rescaled by the similarity on the edges. Thus, the aggregation step is defined as follows:

$$h_{N(v)}^{(l)} \gets \Sigma_{u \in N(v)}({h_u^{(l-1)} * s_{uv}}),$$

where $s_{uv}$ is the similarity score between two vertices $u$ and $v$.

The GNN model has multiple layers. In each layer, a vertex accesses its direct neighbors. When we stack $k$ layers in a model, a node $v$ access neighbors within $k$ hops. The output of the GNN model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

We implement the computation in each layer of the customized GraphSage model in `SAGEConv` and implement the multi-layer model in `GraphSAGEModel`.

In [ ]:
#from sageconv import SAGEConv
from dgl.nn.pytorch import conv as dgl_conv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()
        if n_layers == 1:
            self.layers.append(dgl_conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=None))
        elif n_layers > 1:
            # input layer
            self.layers.append(dgl_conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=activation))
            # hidden layer
            for i in range(n_layers - 2):
                self.layers.append(dgl_conv.SAGEConv(n_hidden, n_hidden, aggregator_type,
                                            feat_drop=dropout, activation=activation))
            # output layer
            self.layers.append(dgl_conv.SAGEConv(n_hidden, out_dim, aggregator_type,
                                        feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h)
            #h = layer(g, h, g.edata['similarity'])
            #h = tmp + prev_h
            #prev_h = h
        return h

## Train Item Embeddings

We train the item embeddings with the edges in the item graph as the training signal. This step is very similar to the link prediction task in the [basic applications](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

Because the MovieLens dataset has sparse features (both genre and title are stored as multi-hot encoding). The sparse features have many dimensions. To run GNN on the item features, we first create an encoding layer to project the sparse features to a lower dimension. 

In [ ]:
def mix_embeddings(h, ndata, emb, proj):
    '''Combine node-specific trainable embedding ``h`` with categorical inputs
    (projected by ``emb``) and numeric inputs (projected by ``proj``).
    '''
    e = []
    for key, value in ndata.items():
        if value.dtype == torch.int64:
            e.append(emb[key](value))
        elif value.dtype == torch.float32:
            e.append(proj[key](value))
    if len(e) == 0:
        return h
    else:
        return h + torch.stack(e, 0).sum(0)
    
class EncodeLayer(nn.Module):
    def __init__(self, ndata, num_hidden, device):
        super(EncodeLayer, self).__init__()
        self.proj = nn.ModuleDict()
        self.emb = nn.ModuleDict()
        for key in ndata.keys():
            vals = ndata[key]
            if vals.dtype == torch.float32:
                self.proj[key] = nn.Sequential(
                                    nn.Linear(ndata[key].shape[1], num_hidden),
                                    nn.LeakyReLU(),
                                    )
            elif vals.dtype == torch.int64:
                self.emb[key] = nn.Embedding(
                            vals.max().item() + 1,
                            num_hidden,
                            padding_idx=0)
                
    def forward(self, ndata):
        return mix_embeddings(0, ndata, self.emb, self.proj)

In [ ]:
class FISMrating(nn.Module):
    r"""
    PinSAGE + FISM for item-based recommender systems
    The formulation of FISM goes as
    .. math::
       r_{ui} = b_u + b_i + \left(n_u^+\right)^{-\alpha}
       \sum_{j \in R_u^+} p_j q_i^\top
    In FISM, both :math:`p_j` and :math:`q_i` are trainable parameters.  Here
    we replace them as outputs from two PinSAGE models ``P`` and
    ``Q``.
    """
    def __init__(self, P, Q, num_users, num_movies, alpha=0):
        super().__init__()

        self.P = P
        self.Q = Q
        self.b_u = nn.Parameter(torch.zeros(num_users))
        self.b_i = nn.Parameter(torch.zeros(num_movies))
        self.alpha = alpha

    
    def forward(self, I, U, I_neg, I_U, N_U):
        '''
        I: 1D LongTensor
        U: 1D LongTensor
        I_neg: 2D LongTensor (batch_size, n_negs)
        '''
        batch_size = I.shape[0]
        device = I.device
        I_U = I_U.to(device)
        # number of interacted items
        N_U = N_U.to(device)
        U_idx = torch.arange(U.shape[0], device=device).repeat_interleave(N_U)

        q = self.Q(I)
        p = self.P(I_U)
        p_self = self.P(I)
        p_sum = torch.zeros_like(q)
        p_sum = p_sum.scatter_add(0, U_idx[:, None].expand_as(p), p)    # batch_size, n_dims
        p_ctx = p_sum - p_self
        pq = (p_ctx * q).sum(1) / ((N_U.float() - 1).clamp(min=1) ** self.alpha)
        r = self.b_u[U] + self.b_i[I] + pq

        if I_neg is not None:
            n_negs = I_neg.shape[1]
            I_neg_flat = I_neg.view(-1)
            q_neg = self.Q(I_neg_flat)
            q_neg = q_neg.view(batch_size, n_negs, -1)  # batch_size, n_negs, n_dims
            pq_neg = (p_ctx.unsqueeze(1) * q_neg).sum(2) / (N_U.float().unsqueeze(1) ** self.alpha)
            r_neg = self.b_u[U].unsqueeze(1) + self.b_i[I_neg] + pq_neg
            return r, r_neg
        else:
            return r

We use the FISM model to train.

In [ ]:
beta = 0
gamma = 0

class FISM(nn.Module):
    def __init__(self, user_movie_spm, gconv_p, gconv_q, g, num_hidden, device):
        super(FISM, self).__init__()
        num_users = user_movie_spm.shape[0]
        num_movies = user_movie_spm.shape[1]
        self.encode_p = EncodeLayer(g.ndata, num_hidden, device)
        self.encode_q = EncodeLayer(g.ndata, num_hidden, device)
        self.gconv_p = gconv_p
        self.gconv_q = gconv_q
        P = lambda I: self.gconv_p(g, self.encode_p(g.ndata))[I]
        Q = lambda I: self.gconv_q(g, self.encode_q(g.ndata))[I]
        self.fism_rating = FISMrating(P, Q, num_users, num_movies, 1)

    def est_rating(self, I, U, I_neg, I_U, N_U):
        r, r_neg = self.fism_rating(I, U, I_neg, I_U, N_U)
        neg_sample_size = int(len(r_neg) / len(r))
        return torch.unsqueeze(r, 1), r_neg.reshape((-1, neg_sample_size))

    def loss(self, r_ui, neg_r_ui):
        diff = 1 - (r_ui - neg_r_ui)
        return torch.sum(torch.mul(diff, diff)/2)# \
        #    + beta/2 * torch.sum(torch.mul(P, P) + torch.mul(Q, Q)) \
        #    + gamma/2 * (torch.sum(torch.mul(self.fism_rating.b_u, self.fism_rating.b_u)) \
        #                 + torch.sum(torch.mul(self.fism_rating.b_i, self.fism_rating.b_i)))

    def forward(self, I, U, I_neg, I_U, N_U):
        r, r_neg = self.fism_rating(I, U, I_neg, I_U, N_U)
        neg_sample_size = int(len(r_neg) / len(r))
        r_neg = r_neg.reshape((-1, neg_sample_size))
        return self.loss(r, r_neg)

In [ ]:
class EdgeSampler:
    def __init__(self, user_movie_spm, batch_size, neg_sample_size):
        edge_ids = np.random.permutation(user_movie_spm.nnz)
        self.batches = np.split(edge_ids, np.arange(batch_size, len(edge_ids), batch_size))
        self.idx = 0
        self.users = user_movie_spm.row
        self.movies = user_movie_spm.col
        self.user_movie_spm = user_movie_spm.tocsr()
        self.num_movies = user_movie_spm.shape[1]
        self.num_users = user_movie_spm.shape[0]
        self.neg_sample_size = neg_sample_size
        
    def __next__(self):
        if self.idx == len(self.batches):
            raise StopIteration
        batch = self.batches[self.idx]
        self.idx += 1
        I_neg = np.random.choice(self.num_movies, len(batch) * self.neg_sample_size)
        I = self.movies[batch]
        U = self.users[batch]
        neighbors = self.user_movie_spm[U]
        I_neg = I_neg.reshape(-1, self.neg_sample_size)
        I = torch.LongTensor(I).to(device)
        U = torch.LongTensor(U).to(device)
        I_neg = torch.LongTensor(I_neg).to(device)
        I_U = torch.LongTensor(neighbors.indices).to(device)
        N_U = torch.LongTensor(neighbors.indptr[1:] - neighbors.indptr[:-1]).to(device)
        return I, U, I_neg, I_U, N_U
    
    def __iter__(self):
        return self

We evaluate the performance of the trained item embeddings in the item-based recommendation task. We use the last item that a user purchased to represent the user and compute the similarity between the last item and a list of items (an item the user will purchase and a set of randomly sampled items). We calculate the ranking of the item that will be purchased among the list of items.

In [ ]:
def RecValid(model, user_movie_spm):
    model.eval()
    with torch.no_grad():
        neg_movies_eval = neg_valid[users_valid]
        neighbors = user_movie_spm.tocsr()[users_valid]
        I_U = torch.LongTensor(neighbors.indices)
        N_U = torch.LongTensor(neighbors.indptr[1:] - neighbors.indptr[:-1])
        r, neg_r = model.est_rating(torch.LongTensor(movies_valid).to(device),
                                    torch.LongTensor(users_valid).to(device),
                                    torch.LongTensor(neg_movies_eval).to(device),
                                    I_U.to(device),
                                    N_U.to(device))
        neg_sample_size = int(len(neg_r) / len(r))
        neg_r = neg_r.reshape((-1, neg_sample_size))
        hits10 = (torch.sum(neg_r >= r, 1) <= 10).cpu().numpy()
        return np.mean(hits10)
    
def RecTest(model, user_movie_spm):
    model.eval()
    with torch.no_grad():
        neg_movies_eval = neg_test[users_test]
        neighbors = user_movie_spm.tocsr()[users_test]
        I_U = torch.LongTensor(neighbors.indices)
        N_U = torch.LongTensor(neighbors.indptr[1:] - neighbors.indptr[:-1])
        r, neg_r = model.est_rating(torch.LongTensor(movies_test).to(device),
                                    torch.LongTensor(users_test).to(device),
                                    torch.LongTensor(neg_movies_eval).to(device),
                                    I_U.to(device),
                                    N_U.to(device))
        neg_sample_size = int(len(neg_r) / len(r))
        neg_r = neg_r.reshape((-1, neg_sample_size))
        hits10 = (torch.sum(neg_r >= r, 1) <= 10).cpu().numpy()
        
        #for popularity, users in test_deg_dict.items():
        #    print(popularity, np.mean(hits10[users]))
        return np.mean(hits10)

Now we put everything in the training loop.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


#Model hyperparameters
n_hidden = 16
n_layers = 4
dropout = 0
aggregator_type = 'gcn'

# create GraphSAGE model
gconv_p = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

gconv_q = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

model = FISM(user_movie_spm, gconv_p, gconv_q, g, n_hidden, device).to(device)
g.to(device)
features = features.to(device)

# Training hyperparameters
weight_decay = 1e-3
n_epochs = 10
lr = 1e-6
neg_sample_size = 20

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

batch_size = 1024
print('#edges:', user_movie_spm.nnz)
print('#batch/epoch:', user_movie_spm.nnz/batch_size)

# initialize graph
dur = []
prev_acc = 0
for epoch in range(n_epochs):
    model.train()
    losses = []
    for I, U, I_neg, I_U, N_U in EdgeSampler(user_movie_spm, batch_size, neg_sample_size):
        loss = model(I, U, I_neg, I_U, N_U)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().item())
    
    hits10 = RecValid(model, user_movie_spm)
    print("Epoch {:05d} | Loss {:.4f} | HITS@10:{:.4f}".format(epoch, np.mean(losses), np.mean(hits10)))
    if prev_acc > hits10:
        break
    prev_acc = hits10

print()
# Let's save the trained node embeddings.
hits10 = RecTest(model, user_movie_spm)
print("Test HITS@10:{:.4f}".format(np.mean(hits10)))